<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20-%20Last%20Step%20_Comparison%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt

#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW.csv')
#y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data.csv')

n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))


#------------------------------------------------------------------------------------------------------

# split a multivariate long sequence into small sequences, and create target value 1 step in right
def split_sequences(sequences, n_steps):
 X, y = list(), list()
 for i in range(len(sequences)):
 # find the end of this pattern
  end_ix = i + n_steps
 # check if we are beyond the dataset
  if end_ix > len(sequences)-1:
    break
 # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_pred, kde=True, color="red")
plt.subplot(1,2,2)

fx_window1 = 5
fx_window2 = 8
fx_window3 = 10
fx_window4 = 12
fx_window5 = 15

#-------------------------Take each lng sequence and loop. Send it to split_sequence with a window size, get smaller slices and target values back.
#-------------------------Then,

def get_mse(sequences,window):
  x_1,y_1 =  split_sequences( sequences,window)
  x_1=x_1.reshape(x_1.shape[0],x_1.shape[1]*x_1.shape[2])
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  regressor = MultiTaskLassoCV(alphas=np.arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  maxval2 = x_1.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  x_train = x_1[0:count_train2]
  x_test = x_1[count_train2:]
  y_train = y_1[0:count_train2]
  y_test = y_1[count_train2:]
  regressor.fit(x_train, y_train)
  ypred = regressor.predict(x_test)
  mse = mean_squared_error(y_test, ypred)
  return mse

rmse_list1 = list()
rmse_list2 = list()
rmse_list3 = list()
rmse_list4 = list()
rmse_list5 = list()
rmse_list_dgr = list()

n_fold = 6


#for i in range(x.shape[0]) :

for i in range(200,300) :
  mse1 = get_mse(x[i],fx_window1)
  rmse_list1.append(mse1)
  mse2 = get_mse(x[i],fx_window2)
  rmse_list2.append(mse2)
  mse3 = get_mse(x[i],fx_window3)
  rmse_list3.append(mse3)
  mse4 = get_mse(x[i],fx_window4)
  rmse_list4.append(mse4)
  mse5 = get_mse(x[i],fx_window5)
  rmse_list5.append(mse5)
  mse_dgr = get_mse(x[i],int(y_pred[i]))
  rmse_list_dgr.append(mse_dgr)

MSE1 = mean(rmse_list1)
MSE2 = mean(rmse_list2)
MSE3 = mean(rmse_list3)
MSE4 = mean(rmse_list4)
MSE5 = mean(rmse_list5)
MSE_DGR = mean(rmse_list_dgr)

RMSE1 = sqrt(MSE1)
RMSE2 = sqrt(MSE2)
RMSE3 = sqrt(MSE3)
RMSE4 = sqrt(MSE4)
RMSE5 = sqrt(MSE5)
RMSE_DGR = sqrt(MSE_DGR)

print (RMSE1,RMSE2,RMSE3,RMSE4,RMSE5, RMSE_DGR)





7.12444206762597e-05 3.9140657348783304e-05 3.347735984398102e-05 2.3212772452279628e-05 1.3871700796718934e-05 4.653186746428274e-05


In [ ]:
from google.colab import drive
drive.mount('/content/drive')